In [1]:
import pandas as pd

In [6]:
register = pd.read_csv("data/서울시 건축물대장 정보.csv", encoding = "cp949", low_memory = False)
pyojebu = pd.read_excel("data/서울시 건축물대장 표제부 정보.xlsm")
chonggual_pyojebu = pd.read_csv("data/서울시 건축물대장 총괄표제부 정보.csv", encoding = "cp949", low_memory = False)
floor_info = pd.read_csv("data/서울특별시_건축물대장_층별개요정보(2019년).csv", encoding = "cp949", low_memory = False)
area_info = pd.read_csv("data/서울시 건축물대장 지역지구구역 정보.csv", encoding = "cp949", low_memory = False)
jeonyubu = pd.read_csv("data/서울시 건축물대장 전유부 정보.csv", encoding = "cp949", low_memory = False)
jeonyubu_area = pd.read_csv("data/서울시 건축물대장 전유공유면적표 정보.csv", encoding = "cp949", low_memory = False)

In [7]:
print(f"--------------------건축물대장 columns name \n {register.columns}")
print(f"--------------------표제부 columns name \n {pyojebu.columns}")
print(f"--------------------총괄표제부 columns name \n {chonggual_pyojebu.columns}")
print(f"--------------------층별개요 columns name \n {floor_info.columns}")
print(f"--------------------지역지구구역 columns name \n {area_info.columns}")
print(f"--------------------전유부 columns name \n {jeonyubu.columns}")
print(f"--------------------전유공유면적표 columns name \n {jeonyubu_area.columns}")

--------------------건축물대장 columns name 
 Index(['관리_건축물대장_PK', '관리_상위_건축물대장_PK', '대장_구분_코드', '대장_종류_코드', '시군구_코드',
       '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '건물_명',
       '위반_건축물_여부', '대장_일련번호', '총괄표제부_일련번호', '표제부_일련번호', '전유부_일련번호',
       '새주소_도로_코드', '새주소_법정동_코드', '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번',
       '작업_일자'],
      dtype='object')
--------------------표제부 columns name 
 Index(['건축물대장 관리번호', '주용도 구분', '기타 용도', '세대수', '가구수', '구조 구분', '기타 구조',
       '지붕 구분', '기타 지붕', '건축 면적', '연 면적', '용적율 산정 연면적', '높이', '대지 면적', '건폐율',
       '용적율', '지상층수', '지하층수', '승용 승강기수', '비상용 승강기수', '부속 건축물수', '부속 건축물 면적',
       '옥내 자주식 대수', '옥내 자주식 면적', '옥외 자주식 대수', '옥외 자주식 면적', '옥내 기계식 대수',
       '옥내 기계식 면적', '옥외 기계식 대수', '옥외 기계식 면적'],
      dtype='object')
--------------------총괄표제부 columns name 
 Index(['건축물대장 관립번호', '대지 면적', '건축 면적', '건폐율', '연면적', '용적율 산정 연면적', '용적율',
       '주건축물수', '부속 건축물수', '부속 건축물 면적', '세대수', '가구수', '총주차수', '옥내 자주식 대수',
       '옥내 자주식 면적', '옥외 자주식 대수', '옥외 자주식 

In [8]:
print(f"--------------------건축물대장 shape \n {register.shape}")
print(f"--------------------표제부 shape \n {pyojebu.shape}")
print(f"--------------------총괄표제부 shape \n {chonggual_pyojebu.shape}")
print(f"--------------------층별개요 shape \n {floor_info.shape}")
print(f"--------------------지역지구구역 shape \n {area_info.shape}")
print(f"--------------------전유부 shape \n {jeonyubu.shape}")
print(f"--------------------전유공유면적표 shape \n {jeonyubu_area.shape}")

--------------------건축물대장 shape 
 (924620, 24)
--------------------표제부 shape 
 (606665, 30)
--------------------총괄표제부 shape 
 (19840, 21)
--------------------층별개요 shape 
 (845040, 13)
--------------------지역지구구역 shape 
 (775035, 6)
--------------------전유부 shape 
 (3626813, 4)
--------------------전유공유면적표 shape 
 (512878, 12)


건축물대장(register) 전처리

In [23]:
register.head()

,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,...,대장_일련번호,총괄표제부_일련번호,표제부_일련번호,전유부_일련번호,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,작업_일자
0,11110-1000000000000001520865,11110-4472,집합,전유부,종로구,원서동,대지,86,20,NaN,...,97,1.0,1.0,97,서울특별시 종로구 창덕궁4길,원서동,지상,9.0,NaN,20230407
1,11215-21131,NaN,일반,일반건축물,광진구,자양동,대지,11,10,NaN,...,10,1.0,10.0,0,NaN,NaN,지상,NaN,NaN,20230407
2,11680-189922,11680-12797,집합,전유부,강남구,역삼동,대지,832,7,NaN,...,86,1.0,1.0,86,서울특별시 강남구 강남대로,역삼동,지상,320.0,NaN,20230407
3,11680-100262797,11680-100262790,집합,전유부,강남구,자곡동,대지,636,0,NaN,...,8,1.0,1.0,8,서울특별시 강남구 자곡로,자곡동,지상,114.0,NaN,20230407
4,11560-135509,11560-35489,집합,전유부,영등포구,대림동,대지,1122,0,NaN,...,150,1.0,1.0,150,서울특별시 영등포구 시흥대로,대림동,지상,589.0,8.0,20230407


In [49]:
register["로트"].describe()

count     962
unique     35
top        32
freq      317
Name: 로트, dtype: object

In [24]:
pre_register = register.drop(["대지_구분_코드", "특수지_명", "블록", "로트", "대장_일련번호", "총괄표제부_일련번호", "표제부_일련번호", "전유부_일련번호", "새주소_법정동_코드", "작업_일자"], axis = 1)
pre_register.head()


,관리_건축물대장_PK,관리_상위_건축물대장_PK,대장_구분_코드,대장_종류_코드,시군구_코드,법정동_코드,번,지,건물_명,위반_건축물_여부,새주소_도로_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번
0,11110-1000000000000001520865,11110-4472,집합,전유부,종로구,원서동,86,20,NaN,NaN,서울특별시 종로구 창덕궁4길,지상,9.0,NaN
1,11215-21131,NaN,일반,일반건축물,광진구,자양동,11,10,NaN,NaN,NaN,지상,NaN,NaN
2,11680-189922,11680-12797,집합,전유부,강남구,역삼동,832,7,황화빌딩,NaN,서울특별시 강남구 강남대로,지상,320.0,NaN
3,11680-100262797,11680-100262790,집합,전유부,강남구,자곡동,636,0,NaN,NaN,서울특별시 강남구 자곡로,지상,114.0,NaN
4,11560-135509,11560-35489,집합,전유부,영등포구,대림동,1122,0,신대림자이,NaN,서울특별시 영등포구 시흥대로,지상,589.0,8.0


표제부(pyojebu) 전처리

In [25]:
pyojebu.head()

,건축물대장 관리번호,주용도 구분,기타 용도,세대수,가구수,구조 구분,기타 구조,지붕 구분,기타 지붕,건축 면적,...,부속 건축물수,부속 건축물 면적,옥내 자주식 대수,옥내 자주식 면적,옥외 자주식 대수,옥외 자주식 면적,옥내 기계식 대수,옥내 기계식 면적,옥외 기계식 대수,옥외 기계식 면적
0,11110-17349,단독주택,"주택, 정자",0,1,벽돌구조,세멘벽돌조,(철근)콘크리트,경사스라브,0.0,...,1,5.76,0,0.0,0,0.0,0,0.0,0,0.0
1,11140-17469,단독주택,주택,0,1,벽돌구조,연와조,기와,도단즙,0.0,...,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0
2,11140-17747,제2종근린생활시설,근린생활시설,0,0,철근콘크리트구조,"철근콘크리트조, 연와조",(철근)콘크리트,스라브,0.0,...,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0
3,11140-18605,단독주택,주택,0,1,일반목구조,목조,기와,와즙,0.0,...,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0
4,11110-17852,단독주택,주택,0,1,일반목구조,목조,기와,와즙,0.0,...,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0


In [26]:
pre_pyojebu = pyojebu.drop(["구조 구분", "기타 구조", "지붕 구분", "기타 지붕", "건축 면적", "연 면적", "용적율 산정 연면적", 
                            "높이", "대지 면적", "부속 건축물수", "부속 건축물 면적", "옥내 자주식 면적", "옥외 자주식 면적", 
                            "옥내 기계식 면적", "옥외 기계식 면적"], axis = 1)
pre_pyojebu.head()

,건축물대장 관리번호,주용도 구분,기타 용도,세대수,가구수,건폐율,용적율,지상층수,지하층수,승용 승강기수,비상용 승강기수,옥내 자주식 대수,옥외 자주식 대수,옥내 기계식 대수,옥외 기계식 대수
0,11110-17349,단독주택,"주택, 정자",0,1,0.0,0.0,2,1,0,0,0,0,0,0
1,11140-17469,단독주택,주택,0,1,0.0,0.0,1,0,0,0,0,0,0,0
2,11140-17747,제2종근린생활시설,근린생활시설,0,0,0.0,0.0,3,1,0,0,0,0,0,0
3,11140-18605,단독주택,주택,0,1,0.0,0.0,1,0,0,0,0,0,0,0
4,11110-17852,단독주택,주택,0,1,0.0,0.0,1,0,0,0,0,0,0,0


총괄표제부(chonggual_pyojebu) 전처리

In [27]:
chonggual_pyojebu.head()

,건축물대장 관립번호,대지 면적,건축 면적,건폐율,연면적,용적율 산정 연면적,용적율,주건축물수,부속 건축물수,부속 건축물 면적,...,가구수,총주차수,옥내 자주식 대수,옥내 자주식 면적,옥외 자주식 대수,옥외 자주식 면적,옥내 기계식 대수,옥내 기계식 면적,옥외 기계식 대수,옥외 기계식 면적
0,11410-100205741,409.9,244.980,59.770,818.36,818.360,199.650,2,0,0.0,...,0,13,7,187.89,6,0.0,0,0.0,0,0.0
1,11410-100206241,4243.0,1167.543,27.517,7811.37,6947.697,163.745,2,0,0.0,...,0,0,0,0.00,47,0.0,0,0.0,0,0.0
2,11410-100206421,426.0,226.170,53.090,848.70,848.700,199.230,2,0,0.0,...,0,12,9,0.00,3,0.0,0,0.0,0,0.0
3,11215-569,0.0,0.000,0.000,102.66,0.000,0.000,2,0,0.0,...,1,0,0,0.00,0,0.0,0,0.0,0,0.0
4,11215-570,0.0,0.000,0.000,80.55,0.000,0.000,2,0,0.0,...,1,0,0,0.00,0,0.0,0,0.0,0,0.0


In [28]:
pre_chonggual_pyojebu = chonggual_pyojebu.drop(
    ["대지 면적", "건축 면적", "연면적", "용적율 산정 연면적", "부속 건축물수", "부속 건축물 면적","총주차수",  "옥내 자주식 면적", 
     "옥외 자주식 면적", "옥내 기계식 면적", "옥외 기계식 면적"], axis = 1
)
pre_chonggual_pyojebu.head()

,건축물대장 관립번호,건폐율,용적율,주건축물수,세대수,가구수,옥내 자주식 대수,옥외 자주식 대수,옥내 기계식 대수,옥외 기계식 대수
0,11410-100205741,59.770,199.650,2,16,0,7,6,0,0
1,11410-100206241,27.517,163.745,2,0,0,0,47,0,0
2,11410-100206421,53.090,199.230,2,15,0,9,3,0,0
3,11215-569,0.000,0.000,2,0,1,0,0,0,0
4,11215-570,0.000,0.000,2,0,1,0,0,0,0


건축물대장 층별개요정보(floor_info) 전처리

In [29]:
floor_info.head()

,관리_층별_개요_PK,관리_건축물대장_PK,관리_주_건축물대장_PK,주_부속_구분_코드,주_부속_일련번호,층_구분_코드,층_번호,층_번호_명,구조_코드,기타_구조,주_용도_코드,기타_용도,면적
0,11110-39088,11110-12767,NaN,0.0,1,20.0,1,1층,11.0,세멘연와조,01001,주택,66.12
1,11110-100140524,11110-100214285,NaN,0.0,1,30.0,1,옥탑1층,21.0,철근콘크리트구조,04402,사무소,46.99
2,11650-18621,11650-4647,NaN,0.0,1,10.0,1,지층,11.0,연와조,01001,NaN,8.56
3,11380-89611,11380-27785,NaN,0.0,1,20.0,3,3층,21.0,철근콘크리트구조,01003,다가구주택,101.84
4,11650-18557,11650-4628,NaN,0.0,1,20.0,2,2층,11.0,연와조,01001,주택,51.90


In [30]:
pre_floor_info = floor_info.drop(["주_부속_일련번호", "구조_코드", "기타_구조"], axis = 1)
# 주_용도_코드: 공통 코드 CM024
pre_floor_info.head()

,관리_층별_개요_PK,관리_건축물대장_PK,관리_주_건축물대장_PK,주_부속_구분_코드,층_구분_코드,층_번호,층_번호_명,주_용도_코드,기타_용도,면적
0,11110-39088,11110-12767,NaN,0.0,20.0,1,1층,01001,주택,66.12
1,11110-100140524,11110-100214285,NaN,0.0,30.0,1,옥탑1층,04402,사무소,46.99
2,11650-18621,11650-4647,NaN,0.0,10.0,1,지층,01001,NaN,8.56
3,11380-89611,11380-27785,NaN,0.0,20.0,3,3층,01003,다가구주택,101.84
4,11650-18557,11650-4628,NaN,0.0,20.0,2,2층,01001,주택,51.90


건축물대장 지역지구구역(area_info) 전처리

In [31]:
area_info.head()

,관리_지역지구구역_pk,관리_건축물대장_pk,지역지구구역_구분_코드,지역지구구역_코드,대표_여부,기타_지역지구구역
0,11110-1,11110-957,용도지역코드,NaN,대표,일반주거지역
1,11110-10,11110-1007,용도지구코드,NaN,대표,최고고도지구
2,11110-100,11110-1206,용도지역코드,일반상업지역,대표,일반상업지역
3,11110-1000,11110-3432,용도지구코드,NaN,NaN,고도최고지구
4,11110-10000,11110-19856,용도지구코드,NaN,대표,주차장정비지구


In [32]:
pre_area_info = area_info.drop(["관리_지역지구구역_pk"], axis = 1)
pre_area_info.head()

,관리_건축물대장_pk,지역지구구역_구분_코드,지역지구구역_코드,대표_여부,기타_지역지구구역
0,11110-957,용도지역코드,NaN,대표,일반주거지역
1,11110-1007,용도지구코드,NaN,대표,최고고도지구
2,11110-1206,용도지역코드,일반상업지역,대표,일반상업지역
3,11110-3432,용도지구코드,NaN,NaN,고도최고지구
4,11110-19856,용도지구코드,NaN,대표,주차장정비지구


전유부(jeonyubu) 전처리

In [33]:
jeonyubu.head() 

,관리_건축물대장_PK,동명칭,호_명,층_구분_코드
0,11110-73094,NaN,B01호,지하
1,11110-60758,사직동다세대주택,301호,지상
2,11110-60761,NaN,401호,지상
3,11530-29716,107동,503호,지상
4,11530-29717,107동,504호,지상


In [34]:
# 할게 없지만 변수명 통일
pre_jeonyubu = jeonyubu

전유공유면적표(jenyubu_area) 전처리

In [35]:
jeonyubu_area.head()

,관리_전유_공용_면적_pk,호별명세_pk,평형_구분_명,전유_공용_구분_코드,주_부속_구분_코드,층_구분_코드,층_번호,구조_코드,주_용도_코드,기타_용도,면적,작업_일자
0,11215-100104314,11215-100100740,B27.91,2.0,0.0,40.0,0,21.0,02003,도시형생활주택(단지형다세대주택(계단실)),4.950,20220722
1,11230-100083014,11230-100094031,502호(103동),2.0,0.0,40.0,0,21.0,02003,계단실,9.296,20220304
2,11260-100084701,11260-100041098,A-501,1.0,0.0,20.0,5,21.0,02003,1세대,47.080,20210414
3,11260-100084705,11260-100041098,A-301,2.0,0.0,40.0,0,21.0,02003,계단실,10.560,20210414
4,11260-100084706,11260-100041098,A-302,2.0,0.0,40.0,0,21.0,02003,계단실,11.540,20210414


In [36]:
pre_jeonyubu_area = jeonyubu_area.drop(["평형_구분_명", "구조_코드", "작업_일자"], axis = 1)
pre_jeonyubu_area.head()

,관리_전유_공용_면적_pk,호별명세_pk,전유_공용_구분_코드,주_부속_구분_코드,층_구분_코드,층_번호,주_용도_코드,기타_용도,면적
0,11215-100104314,11215-100100740,2.0,0.0,40.0,0,02003,도시형생활주택(단지형다세대주택(계단실)),4.950
1,11230-100083014,11230-100094031,2.0,0.0,40.0,0,02003,계단실,9.296
2,11260-100084701,11260-100041098,1.0,0.0,20.0,5,02003,1세대,47.080
3,11260-100084705,11260-100041098,2.0,0.0,40.0,0,02003,계단실,10.560
4,11260-100084706,11260-100041098,2.0,0.0,40.0,0,02003,계단실,11.540


In [40]:
print(f"--------------------전처리 건축물대장 columns name \n {pre_register.columns}")
print(f"--------------------전처리 표제부 columns name \n {pre_pyojebu.columns}")
print(f"--------------------전처리 총괄표제부 columns name \n {pre_chonggual_pyojebu.columns}")
print(f"--------------------전처리 층별개요 columns name \n {pre_floor_info.columns}")
print(f"--------------------전처리 지역지구구역 columns name \n {pre_area_info.columns}")
print(f"--------------------전처리 전유부 columns name \n {pre_jeonyubu.columns}")
print(f"--------------------전처리 전유공유면적표 columns name \n {pre_jeonyubu_area.columns}")

--------------------전처리 건축물대장 columns name 
 Index(['관리_건축물대장_PK', '관리_상위_건축물대장_PK', '대장_구분_코드', '대장_종류_코드', '시군구_코드',
       '법정동_코드', '번', '지', '건물_명', '위반_건축물_여부', '새주소_도로_코드', '새주소_지상지하_코드',
       '새주소_본_번', '새주소_부_번'],
      dtype='object')
--------------------전처리 표제부 columns name 
 Index(['건축물대장 관리번호', '주용도 구분', '기타 용도', '세대수', '가구수', '건폐율', '용적율', '지상층수',
       '지하층수', '승용 승강기수', '비상용 승강기수', '옥내 자주식 대수', '옥외 자주식 대수', '옥내 기계식 대수',
       '옥외 기계식 대수'],
      dtype='object')
--------------------전처리 총괄표제부 columns name 
 Index(['건축물대장 관립번호', '건폐율', '용적율', '주건축물수', '세대수', '가구수', '옥내 자주식 대수',
       '옥외 자주식 대수', '옥내 기계식 대수', '옥외 기계식 대수'],
      dtype='object')
--------------------전처리 층별개요 columns name 
 Index(['관리_층별_개요_PK', '관리_건축물대장_PK', '관리_주_건축물대장_PK', '주_부속_구분_코드', '층_구분_코드',
       '층_번호', '층_번호_명', '주_용도_코드', '기타_용도', '면적'],
      dtype='object')
--------------------전처리 지역지구구역 columns name 
 Index(['관리_건축물대장_pk', '지역지구구역_구분_코드', '지역지구구역_코드', '대표_여부', '기타_지역지구구역'], dtype='objec

In [41]:
print(f"--------------------건축물대장 shape \n {pre_register.shape}")
print(f"--------------------표제부 shape \n {pre_pyojebu.shape}")
print(f"--------------------총괄표제부 shape \n {pre_chonggual_pyojebu.shape}")
print(f"--------------------층별개요 shape \n {pre_floor_info.shape}")
print(f"--------------------지역지구구역 shape \n {pre_area_info.shape}")
print(f"--------------------전유부 shape \n {pre_jeonyubu.shape}")
print(f"--------------------전유공유면적표 shape \n {pre_jeonyubu_area.shape}")

--------------------건축물대장 shape 
 (924620, 14)
--------------------표제부 shape 
 (606665, 15)
--------------------총괄표제부 shape 
 (19840, 10)
--------------------층별개요 shape 
 (845040, 10)
--------------------지역지구구역 shape 
 (775035, 5)
--------------------전유부 shape 
 (3626813, 4)
--------------------전유공유면적표 shape 
 (512878, 9)
